In [1]:
using Pkg
Pkg.gc()

      Active manifest files: 1 found
      Active artifact files: 119 found
      Active scratchspaces: 6 found
     Deleted no artifacts, repos, packages or scratchspaces


In [ ]:
Pkg.add("Graphs")
Pkg.add("GraphPlot")
Pkg.add("Plots")

In [2]:
using Graphs
using GraphPlot
using Plots

[ Info: Precompiling GraphPlot [a2cc645c-3eea-5389-862e-a155d0052231]
[ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
[ Info: Precompiling IJuliaExt [2f4121a4-3b3a-5ce6-9c5e-1f2673ce168a]


In [ ]:
# Function to parse the tree structure
function parse_tree_data(file_path::String)
    nodes = Dict{String, Vector{String}}()  # Dictionary to hold parent-child relationships
    parents = Dict{String, String}()         # To track parent nodes
    
    open(file_path, "r") do file
        for line in eachline(file
            if startswith(line, "SymbolicPlanners.PathNode")
                # Extract node identifier (e.g., 0xa084eaed7be71401)
                node_id = match(r"0x[a-fA-F0-9]+", line).match
                # Extract parent node id
                parent_id_match = match(r"LinkedNodeRef\((0x[a-fA-F0-9]+)", line)
                if parent_id_match !== nothing
                    parent_id = parent_id_match.captures[1]
                    push!(nodes[parent_id], node_id)
                    parents[node_id] = parent_id
                else
                    # Handle root node case if needed
                    push!(nodes["root"], node_id)
                end
            end
        end
    end
    
    return nodes, parents
end

In [ ]:
# Function to create a graph from the parsed data
function create_graph(nodes::Dict{String, Vector{String}}, parents::Dict{String, String})
    g = SimpleDiGraph()
    
    # Add vertices
    for node_id in keys(nodes)
        add_vertex!(g, node_id)
    end
    
    # Add edges
    for (parent, children) in nodes
        for child in children
            add_edge!(g, parent, child)
        end
    end
    
    return g
end

In [ ]:
# Function to plot the graph and save it as a PNG file
function plot_graph(g::SimpleDiGraph, output_file::String)
    # Set the layout and colors
    layout = GraphPlot.dotlayout(g)
    plot(g, layout=layout, node_color=:lightblue, edge_color=:gray, 
         title="Tree Structure", markersize=10)
    
    savefig(output_file)
end


In [ ]:
# Main function to orchestrate the process
function main(file_path::String, output_file::String)
    nodes, parents = parse_tree_data(file_path)
    g = create_graph(nodes, parents)
    plot_graph(g, output_file)
end

In [ ]:
# Call the main function with your file path and desired output file
main("tree_data.txt", "tree_graph.png")